# __Latent Graph Inference__

Es el proceso de aprender o deducir la estructura de un grafo implícito a partir de datos en los que las relaciones explícitas entre las entidades (nodos) no están directamente disponibles. En este contexto, un grafo latente se refiere a una representación de las relaciones subyacentes que se infieren durante el análisis o entrenamiento de un modelo, en lugar de ser proporcionadas como entrada.

----

## **Conceptos Básicos**
1. Nodos: Representan entidades o elementos en los datos (por ejemplo, personas, palabras, genes).
2. Aristas (Edges): Representan las relaciones entre nodos. ¡Pero! **En un grafo latente, estas relaciones no están explícitas, sino que se infieren a partir de patrones en los datos**.
3. Matriz de Adyacencia:
    * En un grafo explícito, es una matriz $A$ donde $A_{ij} = 1$ indica una conexión entre los nodos $i$ y $j$.
    * **En la inferencia de grafos latentes, esta matriz es aprendida durante el entrenamiento del modelo.**

---

## **Objetivo**

El objetivo de la inferencia de grafos latentes es **descubrir relaciones implícitas** que pueden proporcionar *información adicional* para tareas de aprendizaje como clasificación, predicción o agrupamiento.

Por ejemplo:
* En redes sociales, infiere relaciones ocultas basadas en intereses comunes o comportamientos similares.
* En biología, identifica interacciones no observadas entre proteínas o genes.
* En NLP, extrae dependencias implícitas entre palabras o conceptos.

---

## **Métodos Usados para Inferir Grafos**
1. **Graph Neural Networks (GNNs):**
    * Utilizan representaciones latentes de nodos y aprenden conexiones adaptativas entre ellos.
2. **Variational Graph Autoencoders (VGAE):**
    * Utilizan técnicas de autoencoders variacionales para aprender tanto representaciones de nodos como relaciones implícitas (aristas).
3. **Graph Attention Networks (GAT):**
    * Introducen atención para ponderar la importancia de las conexiones entre nodos.
4. **Métodos Probabilísticos:**
    * Modelos como Probabilistic Graphical Models (PGM) y enfoques bayesianos infieren relaciones con base en distribuciones de probabilidad.
5. **Contrastive Learning:**
    * Aprende representaciones de grafos al maximizar la similitud entre nodos conectados implícitamente.


---
### **1. Graph Neural Networks (GNNs)**
Los **GNNs** son redes neuronales diseñadas específicamente para datos en forma de grafos. En el contexto de **Latent Graph Inference**, los GNNs pueden aprender no solo representaciones latentes de los nodos, sino también inferir relaciones adaptativas (es decir, las conexiones o pesos de las aristas).

#### **Componentes principales:**
1. **Propagación de mensajes:**
    * Los nodos intercambian información con sus vecinos.
    * Cada nodo $v$ actualiza su representación usando un agregador de las características de sus vecinos $(N(v))$ y sus propias características:
$$
h_i^{(k+1)} = \text{AGGREGATE}(h_u^{(k)},\forall u \in N(v))
$$
    * Ejemplo de agregador: suma, promedio o función de atención.
2. **Representaciones latentes:**
    * Al final del entrenamiento, cada nodo tiene una representación $h_v$ en un espacio latente que captura sus relaciones y contexto local en el grafo.
  
#### **Ventaja en Latent Graph Inference:**
Si el grafo original no está explícito, los GNN pueden comenzar con un grafo completamente conectado (conexiones densas) y aprender qué conexiones son importantes ajustando los pesos de las aristas.

#### **Variantes de GNNs:**
1. **Graph Convolutional Networks (GCNs):**
    * Introducidas por Kipf y Welling (2017).
    * Usan convoluciones espectrales para mezclar información de nodos conectados:
$$
H^{(k+1)} = \sigma(\hat{D}^{−1/2}\hat{A}\hat{D}^{−1/2}H^{(k)}W^{(k)})
$$
Donde $\hat{A}$ es la matriz de adyacencia normalizada y $W$ son pesos aprendibles.
2. **Graph Attention Networks (GATs):**
    * Asignan diferentes pesos a las conexiones de un nodo utilizando un mecanismo de atención.
    * Calculan coeficientes de atención $\alpha_{ij}$ entre un nodo $i$ y sus vecinos $j$:
$$
\alpha_{ij} = \frac{\exp(\text{LeakyReLU}(a^T[Wh_i||Wh_j]))}{\sum_{k\in N(i)}\exp(\text{LeakyReLU}(a^T[Wh_i||Wh_k]))}
$$


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Datos de ejemplo: 4 nodos, 2 características por nodo
x = torch.tensor([[1, 2], [2, 3], [3, 1], [4, 5]], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 2, 3], [1, 0, 3, 2]], dtype=torch.long)  # Aristas bidireccionales

# Crear grafo
data = Data(x=x, edge_index=edge_index)

# Modelo GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 4)
        self.conv2 = GCNConv(4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(in_channels=2, out_channels=2)
output = model(data)
output


tensor([[-0.5992, -0.7969],
        [-0.5992, -0.7969],
        [-0.8469, -0.5599],
        [-0.8469, -0.5599]], grad_fn=<LogSoftmaxBackward0>)

In [8]:
#pip install torch-geometric
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Cargar el dataset Cora
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Modelo GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Entrenamiento del modelo
model = GCN(in_channels=dataset.num_features, out_channels=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

def test():
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = correct / data.test_mask.sum()
    return acc

for epoch in range(200):
    train()
    acc = test()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Test Accuracy: {acc:.4f}')

Epoch 0, Test Accuracy: 0.5350
Epoch 10, Test Accuracy: 0.7990
Epoch 20, Test Accuracy: 0.7990
Epoch 30, Test Accuracy: 0.7950
Epoch 40, Test Accuracy: 0.8000
Epoch 50, Test Accuracy: 0.7990
Epoch 60, Test Accuracy: 0.8050
Epoch 70, Test Accuracy: 0.8080
Epoch 80, Test Accuracy: 0.8120
Epoch 90, Test Accuracy: 0.8120
Epoch 100, Test Accuracy: 0.8130
Epoch 110, Test Accuracy: 0.8120
Epoch 120, Test Accuracy: 0.8120
Epoch 130, Test Accuracy: 0.8070
Epoch 140, Test Accuracy: 0.8060
Epoch 150, Test Accuracy: 0.8040
Epoch 160, Test Accuracy: 0.8030
Epoch 170, Test Accuracy: 0.8020
Epoch 180, Test Accuracy: 0.8050
Epoch 190, Test Accuracy: 0.8060


---
### **2. Variational Graph Autoencoders (VGAEs)**
Los **VGAEs** son un enfoque específico para inferir grafos. Utilizan técnicas de autoencoders variacionales para aprender tanto representaciones de nodos como la matriz de adyacencia implícita.

#### **Arquitectura:**
1. Encoder:
    * Una red neuronal (como un GCN) mapea las características de los nodos a un espacio latente:
$$
Z = \text{GCN}(X,A)
$$
Donde $Z$ son las representaciones latentes, $X$ las características iniciales y $A$ la matriz de adyacencia (que puede ser inicial o inferida).
2. Decoder:
    * Reconstruye la matriz de adyacencia $\hat{A}$ usando un producto interno en el espacio latente:
$$
\hat{A}_{ij} = \sigma(Z_i^TZ_j)
$$
3. Pérdida:
    * Combina una pérdida de reconstrucción (para $A$) y una pérdida de regularización (KL-divergence):
$$
\mathcal{L} = \mathbb{E}_{q(Z|X,A)}[\log p(A|Z)] - \text{KL}(q(Z|X,A)||p(Z))
$$

#### **Ventaja en Latent Graph Inference:**
Pueden inferir conexiones incluso con datos incompletos, proporcionando una matriz de adyacencia plausible al decodificar.


In [3]:
from torch_geometric.nn import VGAE, GCNConv

# Encoder para VGAE
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

# Datos de ejemplo
x = torch.tensor([[1, 2], [2, 3], [3, 1], [4, 5]], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 2, 3], [1, 0, 3, 2]], dtype=torch.long)  # Aristas bidireccionales

# VGAE Model
model = VGAE(Encoder(in_channels=2, out_channels=2))
z = model.encode(x, edge_index)
reconstructed = model.decode_all(z)
print("Latent Representations (z):", z)
print("Reconstructed Adjacency Matrix:", reconstructed)


ValueError: too many values to unpack (expected 2)

In [9]:
from torch_geometric.nn import VGAE, GCNConv
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F

# Cargar dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Definir Encoder para VGAE
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv_mu = GCNConv(16, out_channels)
        self.conv_logvar = GCNConv(16, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)
        return mu, logvar

# Inicializar VGAE
model = VGAE(Encoder(dataset.num_features, 2))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Función de entrenamiento
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)
    loss = model.recon_loss(z, data.edge_index)
    kl_loss = model.kl_loss()
    loss = loss + 0.01 * kl_loss
    loss.backward()
    optimizer.step()

# Entrenamiento del modelo
for epoch in range(200):
    train()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}')


Epoch 0
Epoch 10
Epoch 20
Epoch 30
Epoch 40
Epoch 50
Epoch 60
Epoch 70
Epoch 80
Epoch 90
Epoch 100
Epoch 110
Epoch 120
Epoch 130
Epoch 140
Epoch 150
Epoch 160
Epoch 170
Epoch 180
Epoch 190


---
### **3. Graph Attention Networks (GATs)**
Los **GATs** son una mejora de los GCNs que introducen atención para aprender la importancia relativa de las conexiones entre nodos.

#### **Características principales:**
1. Mecanismo de atención:
    * Cada nodo evalúa la importancia de sus vecinos usando coeficientes de atención $\alpha_{ij}$, que se calculan para cada arista:
$$
\alpha_{ij} = \text{softmax}_j(e_{ij}), e_{ij} = \text{LeakyReLU}(a^Tf[Wh_i||Wh_j])
$$
Donde $a$ es un vector de pesos de atención aprendible.
2. Actualización de nodos:
    * La representación de cada nodo se actualiza como una combinación ponderada de las características de sus vecinos:
$$
h_i^{(k+1)} = \sigma(\sum_{i\in N(i)} \alpha_{ij}Wh_j)
$$
#### **Ventaja en Latent Graph Inference:**
* Permiten inferir grafos más precisos al enfocarse en conexiones relevantes y descartar las irrelevantes.


In [4]:
from torch_geometric.nn import GATConv

# Datos de ejemplo
x = torch.tensor([[1, 2], [2, 3], [3, 1], [4, 5]], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 2, 3], [1, 0, 3, 2]], dtype=torch.long)

# Modelo GAT
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=4, concat=True)
        self.conv2 = GATConv(8 * 4, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        return F.log_softmax(self.conv2(x, edge_index), dim=1)

model = GAT(in_channels=2, out_channels=2)
output = model(x, edge_index)
print(output)


tensor([[-0.1587, -1.9189],
        [-0.1587, -1.9189],
        [-0.0534, -2.9574],
        [-0.0534, -2.9574]], grad_fn=<LogSoftmaxBackward0>)


In [10]:
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv
import torch
import torch.nn.functional as F

# Cargar dataset Citeseer
dataset = Planetoid(root='/tmp/Citeseer', name='Citeseer')
data = dataset[0]

# Definir modelo GAT
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=8, concat=True)
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.conv1(x, edge_index))
        return F.log_softmax(self.conv2(x, edge_index), dim=1)

# Modelo y optimizador
model = GAT(in_channels=dataset.num_features, out_channels=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

# Entrenamiento
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

def test():
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = correct / data.test_mask.sum()
    return acc

for epoch in range(200):
    train()
    acc = test()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Test Accuracy: {acc:.4f}')


Processing...
Done!


Epoch 0, Test Accuracy: 0.5870
Epoch 10, Test Accuracy: 0.6660
Epoch 20, Test Accuracy: 0.6760
Epoch 30, Test Accuracy: 0.6760
Epoch 40, Test Accuracy: 0.6820
Epoch 50, Test Accuracy: 0.6920
Epoch 60, Test Accuracy: 0.6920
Epoch 70, Test Accuracy: 0.6960
Epoch 80, Test Accuracy: 0.6920
Epoch 90, Test Accuracy: 0.6910
Epoch 100, Test Accuracy: 0.6850
Epoch 110, Test Accuracy: 0.6840
Epoch 120, Test Accuracy: 0.6830
Epoch 130, Test Accuracy: 0.6820
Epoch 140, Test Accuracy: 0.6810
Epoch 150, Test Accuracy: 0.6840
Epoch 160, Test Accuracy: 0.6840
Epoch 170, Test Accuracy: 0.6830
Epoch 180, Test Accuracy: 0.6830
Epoch 190, Test Accuracy: 0.6880


---
### **4. Métodos Probabilísticos**

Estos métodos infieren grafos latentes desde un enfoque estadístico, usando distribuciones de probabilidad para modelar las relaciones.

#### **Ejemplo: Probabilistic Graphical Models (PGMs):**
1. **Modelo básico:**
    * Representan relaciones condicionales entre nodos usando nodos y aristas.
    * Ejemplo: Redes bayesianas o modelos de Markov.
2. **Inferencia:**
    * Utilizan técnicas como inferencia variacional o Gibbs Sampling para inferir conexiones probables entre nodos.

#### **Ventaja en Latent Graph Inference:**
Son ideales cuando se dispone de información previa probabilística sobre los nodos o sus posibles relaciones.


In [11]:
#pip install pgmpy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.8/170.8 kB 699.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 3.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 20.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 kB 5.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 35.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 11.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.7/221.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.4 MB/s 

In [12]:
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination

# Crear un modelo gráfico bayesiano
model = BayesianModel([('A', 'B'), ('B', 'C')])

# Definir distribuciones de probabilidad
from pgmpy.factors.discrete import TabularCPD
cpd_a = TabularCPD(variable='A', variable_card=2, values=[[0.6], [0.4]])
cpd_b = TabularCPD(variable='B', variable_card=2,
                   values=[[0.7, 0.2], [0.3, 0.8]],
                   evidence=['A'], evidence_card=[2])
cpd_c = TabularCPD(variable='C', variable_card=2,
                   values=[[0.9, 0.5], [0.1, 0.5]],
                   evidence=['B'], evidence_card=[2])

# Asociar CPDs al modelo
model.add_cpds(cpd_a, cpd_b, cpd_c)

# Inferencia
infer = VariableElimination(model)
print(infer.query(['C'], evidence={'A': 1}))


INFO:numexpr.utils:Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/opt/conda/envs/umap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.5800 |
+------+----------+
| C(1) |   0.4200 |
+------+----------+


In [13]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Crear modelo
model = BayesianNetwork([('A', 'B'), ('B', 'C')])

# Definir CPDs
cpd_a = TabularCPD(variable='A', variable_card=2, values=[[0.6], [0.4]])
cpd_b = TabularCPD(variable='B', variable_card=2, values=[[0.7, 0.2], [0.3, 0.8]], evidence=['A'], evidence_card=[2])
cpd_c = TabularCPD(variable='C', variable_card=2, values=[[0.9, 0.5], [0.1, 0.5]], evidence=['B'], evidence_card=[2])

model.add_cpds(cpd_a, cpd_b, cpd_c)

# Inferencia
infer = VariableElimination(model)
print(infer.query(['C'], evidence={'A': 1}))


+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.5800 |
+------+----------+
| C(1) |   0.4200 |
+------+----------+


---
### **5. Contrastive Learning**
Este enfoque aprende grafos latentes maximizando la similitud entre nodos conectados implícitamente y minimizando la similitud entre nodos no conectados.

#### **Ejemplo: Graph Contrastive Learning (GCL):**
1. Objetivo:
    * Maximizar la similitud entre nodos $u, v$ si $u∼v$ y minimizarla si no están conectados:
$$
\mathcal{L}_{contrastive} = -\log \frac{\exp(\text{sim}(h_u, h_v))}{\sum_k \exp(\text{sim}(h_u, h_k))}
$$

2. Similitud:
    * Se mide con funciones como el producto punto o la distancia euclidiana.

#### **Ventaja en Latent Graph Inference:**
Aprende representaciones robustas que reflejan relaciones implícitas, incluso con ruido en los datos.


In [14]:
# pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
from torch.nn import CosineSimilarity

# Representaciones de ejemplo
h1 = torch.tensor([[1.0, 0.5], [0.5, 0.3]], dtype=torch.float)
h2 = torch.tensor([[1.1, 0.4], [0.6, 0.2]], dtype=torch.float)

# Similitudes positivas (entre nodos conectados)
similarity = CosineSimilarity(dim=1)(h1, h2)

# Pérdida contrastiva
negative_samples = torch.tensor([[0.1, 0.9], [0.2, 0.8]], dtype=torch.float)
negative_similarity = CosineSimilarity(dim=1)(h1, negative_samples)

loss = -torch.log(torch.exp(similarity) / (torch.exp(similarity) + torch.exp(negative_similarity)))
print("Contrastive Loss:", loss.mean())


Contrastive Loss: tensor(0.5304)


In [15]:
import torch
from torch.nn import CosineSimilarity

# Representaciones de ejemplo
h1 = torch.tensor([[1.0, 0.5], [0.5, 0.3]], dtype=torch.float)
h2 = torch.tensor([[1.1, 0.4], [0.6, 0.2]], dtype=torch.float)

# Similitudes positivas (entre nodos conectados)
similarity = CosineSimilarity(dim=1)(h1, h2)

# Pérdida contrastiva
negative_samples = torch.tensor([[0.1, 0.9], [0.2, 0.8]], dtype=torch.float)
negative_similarity = CosineSimilarity(dim=1)(h1, negative_samples)

loss = -torch.log(torch.exp(similarity) / (torch.exp(similarity) + torch.exp(negative_similarity)))
print("Contrastive Loss:", loss.mean())


Contrastive Loss: tensor(0.5304)


## **Resumen Comparativo**
| Método | Ventajas | Desventajas |
|---|---|---|
| GNNs | Escalables, versátiles | Menos específicas para grafos latentes |
| VGAEs | Dedican atención explícita a inferir grafos | Costo computacional alto |
| GATs | Ponderan relaciones adaptativamente | Complejidad adicional por atención |
| Métodos Probabilísticos | Basados en principios estadísticos sólidos | Requieren suposiciones probabilísticas |
| Contrastive Learning | Robusto contra ruido | Sensible a la elección de datos negativos |

### **Pasos en Latent Graph Inference**
1. **Preparación de los Datos:**
    * Los datos pueden ser características de nodos sin un grafo explícito.
2. **Modelo:**
    * Diseñar un modelo capaz de aprender una estructura latente de grafo (por ejemplo, GNN o VGAE).
3. **Inferencia:**
    * Durante el entrenamiento, se optimiza una representación latente de los nodos y una matriz de adyacencia que capture las relaciones implícitas.
4. **Evaluación:**
    * La calidad del grafo inferido puede evaluarse comparándolo con datos reales (si existen) o evaluando el rendimiento en una tarea específica (como clasificación de nodos).
  
---

### **Aplicaciones**
* **Biología:** Inferir redes genéticas, interacciones proteína-proteína.
* **Redes Sociales:** Detectar comunidades ocultas o relaciones implícitas entre usuarios.
* **Procesamiento de Lenguaje Natural:** Extraer dependencias semánticas entre palabras o conceptos.
* **Sistemas de Recomendación:** Inferir conexiones entre usuarios y productos.